In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

# Questions:
> * 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
> * 2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
> * 3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
> * 4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?
> * 5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?
> * 6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
> * 7. Which lessons are least accessed?
> * 8. Anything else I should be aware of?


In [10]:
cohorts = pd.read_csv('cohorts_data.csv')
logs = pd.read_csv('logs_data.csv')

for col in cohorts.columns:
    if 'Unnamed' in col:
        cohorts = cohorts.drop(columns=[col])

for col in logs.columns:
    if 'Unnamed' in col:
        logs = logs.drop(columns=[col])

cohorts.head()

,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,1,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
1,2,Badlands,#badlands,2014-06-04,2014-08-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2,3,Carlsbad,#carlsbad,2014-09-04,2014-11-05,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
3,4,Denali,#denali,2014-10-20,2015-01-18,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
4,5,Everglades,#everglades,2014-11-18,2015-02-24,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1


In [15]:
cohorts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          53 non-null     int64  
 1   name        53 non-null     object 
 2   slack       53 non-null     object 
 3   start_date  53 non-null     object 
 4   end_date    53 non-null     object 
 5   created_at  53 non-null     object 
 6   updated_at  53 non-null     object 
 7   deleted_at  0 non-null      float64
 8   program_id  53 non-null     int64  
dtypes: float64(1), int64(2), object(6)
memory usage: 3.9+ KB


In [16]:
cohorts= cohorts.drop(columns=['deleted_at'])
cohorts.head(3)

,id,name,slack,start_date,end_date,created_at,updated_at,program_id
0,1,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,1
1,2,Badlands,#badlands,2014-06-04,2014-08-22,2016-06-14 19:52:26,2016-06-14 19:52:26,1
2,3,Carlsbad,#carlsbad,2014-09-04,2014-11-05,2016-06-14 19:52:26,2016-06-14 19:52:26,1


In [70]:
cohorts.tail(4)

,id,name,slack,start_date,end_date,created_at,updated_at,program_id
49,136,Placeholder for students in transition,#null,2021-03-03,2029-03-01,2021-03-03 21:50:49,2021-03-03 21:52:56,2
50,137,Florence,#florence,2021-03-15,2021-09-03,2021-03-15 18:18:20,2021-03-15 18:18:20,3
51,138,Neptune,#neptune,2021-03-15,2021-09-03,2021-03-15 19:57:09,2021-03-15 19:57:09,2
52,139,Oberon,#oberon,2021-04-12,2021-10-01,2021-04-12 18:07:21,2021-04-12 18:07:21,2


In [11]:
logs.head()

,date,time,path,user_id,cohort_id,ip
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


In [17]:
logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900223 entries, 0 to 900222
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   date       900223 non-null  object 
 1   time       900223 non-null  object 
 2   path       900222 non-null  object 
 3   user_id    900223 non-null  int64  
 4   cohort_id  847330 non-null  float64
 5   ip         900223 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 41.2+ MB


In [19]:
logs.cohort_id.value_counts(dropna=False)

28.0     84031
NaN      52893
33.0     40730
29.0     38096
62.0     37109
53.0     36902
24.0     35636
57.0     33844
56.0     33568
51.0     32888
59.0     32015
22.0     30926
58.0     29855
32.0     29356
23.0     28534
52.0     28033
26.0     27749
34.0     26538
25.0     25586
31.0     25359
132.0    23691
55.0     21582
27.0     20743
61.0     17713
134.0    16623
135.0    16397
133.0    14715
14.0      9587
1.0       8890
137.0     8562
21.0      7444
138.0     7276
17.0      4954
13.0      2845
18.0      2158
8.0       1712
139.0     1672
19.0      1237
16.0       755
15.0       691
7.0        598
12.0       302
11.0       253
2.0         93
6.0         72
9.0          5
4.0          4
5.0          1
Name: cohort_id, dtype: int64

In [22]:
logs = logs.fillna(0)

In [72]:
df = pd.merge(left_on=logs.cohort_id, right_on=cohorts.id, left=logs, right=cohorts,how='outer').drop(columns='key_0')

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 900229 entries, 0 to 900228
Data columns (total 14 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        900223 non-null  object 
 1   time        900223 non-null  object 
 2   path        900223 non-null  object 
 3   user_id     900223 non-null  float64
 4   cohort_id   900223 non-null  float64
 5   ip          900223 non-null  object 
 6   id          847336 non-null  float64
 7   name        847336 non-null  object 
 8   slack       847336 non-null  object 
 9   start_date  847336 non-null  object 
 10  end_date    847336 non-null  object 
 11  created_at  847336 non-null  object 
 12  updated_at  847336 non-null  object 
 13  program_id  847336 non-null  float64
dtypes: float64(4), object(10)
memory usage: 103.0+ MB


In [31]:
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,program_id
0,2018-01-26,09:55:03,/,1.0,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0
1,2018-01-26,09:56:02,java-ii,1.0,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1.0,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1.0,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0
4,2018-01-26,10:40:15,javascript-i/functions,1.0,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0


### 7. Which lessons are least accessed?

> * General start and end of entire site
> * General start and end for each module/lesson
> * See if things haven't been accessed in a long time

MVP: 
Lessons


**NOTE:** At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?

In [73]:
#Range of Dates for entire site:
df.date = pd.to_datetime(df.date+" " +df.time)
df[df.date.isnull()].shape[0]

6

> **note:** Only 6 datetime values that are non-functional

In [77]:
# name of the cohorts/placeholders with inaccurate date values
df.name[df.date.isnull()]

900223                                  Carlsbad
900224                                    Balboa
900225                                   Redwood
900226                                       Ada
900227                                        Io
900228    Placeholder for students in transition
Name: name, dtype: object

In [80]:
# confirming the number of occurrences of the above cohorts/ph
df.name.value_counts().tail(10)

Franklin                                  72
Apollo                                     5
Denali                                     4
Redwood                                    1
Io                                         1
Ada                                        1
Everglades                                 1
Balboa                                     1
Carlsbad                                   1
Placeholder for students in transition     1
Name: name, dtype: int64

In [83]:
df.date.min(), df.date.max()

(Timestamp('2018-01-26 09:55:03'), Timestamp('2021-04-21 16:44:39'))

**Conclusion**: Our data started on Jan 26th, 2018 and is current up to April 21, 2021

In [33]:
df.path.value_counts()

/                                                 50313
search/search_index.json                          19519
javascript-i                                      18983
toc                                               18297
java-iii                                          13733
                                                  ...  
13-storytelling/2.2-create                            1
2-sql/database-design                                 1
4-stats/1-descriptive-stats                           1
4-stats/2.1-probability_and_event_interactions        1
where                                                 1
Name: path, Length: 2314, dtype: int64

In [36]:
num_paths = df.path.nunique()

In [167]:
bottom_500 = np.unique(df.path.value_counts().iloc[num_paths-450:,].values)
bottom_500

array([1, 2])

# CONCLUSION:

> * **450** of the **2300** **(~20%)** pages have been accessed only once

In [142]:
word_counts = {}


for path in df.path.value_counts().index:
    word = str(path)
    for elem in word.split('/'):
        if len(elem) > 0 and elem not in list(word_counts.keys()):
            word_counts[elem] = 1
        elif len(elem) > 0:
            word_counts[elem] += 1
            
word_count = pd.Series(word_counts, name='word_count').sort_values(ascending=False)
word_count

content                        337
appendix                       222
html-css                        77
mysql                           77
examples                        77
                              ... 
13.3_Refine                      1
bootstrap-introduction.html      1
styling-webpages                 1
9.20_Data                        1
2-sql                            1
Name: word_count, Length: 1614, dtype: int64

In [128]:
df.path.head()

0                                      /
1                                java-ii
2    java-ii/object-oriented-programming
3     slides/object_oriented_programming
4                 javascript-i/functions
Name: path, dtype: object

In [159]:
first_word = [str(path).split('/')[0] for path in df.path.value_counts().index]
second_word = [str(path).split('/')[1] for path in df.path.value_counts().index if len(str(path).split('/'))>1]
third_word = [str(path).split('/')[2] for path in df.path.value_counts().index if len(str(path).split('/'))>2]
fourth_word = [str(path).split('/')[3] for path in df.path.value_counts().index if len(str(path).split('/'))>3]
fifth_word = [str(path).split('/')[4] for path in df.path.value_counts().index if len(str(path).split('/'))>4]
sixth_word = [str(path).split('/')[5] for path in df.path.value_counts().index if len(str(path).split('/'))>5]
seventh_word = [str(path).split('/')[6] for path in df.path.value_counts().index if len(str(path).split('/'))>6]

word_list = [first_word, second_word, third_word, fourth_word, fifth_word, sixth_word, seventh_word]

def flatten(a_list):
    return [word for words in word_list for word in words]

separators = ['.', '-', '_']
set(flatten(word_list))        

{'',
 'quiz118',
 '3.6-creating-stories',
 'resource-controllers.html',
 'florence-python-assessment.html',
 'MachineLearning_CheatSheet',
 ':view-page',
 '3-prep',
 'css-practice',
 'Regression_Python',
 '10.02.04_EntityLabeling',
 'imports',
 '2.03_Inferential',
 'phpinfo',
 '4-stats',
 'dom-query-modify.html',
 'images',
 'intellij',
 'Probability.md',
 'www.openstreetmap.org',
 '3.1-mysql-introduction',
 '10.04.03_SentimentAnalysis',
 'SpecRunner.html',
 'basic_challenges',
 'flexbox-cards.zip',
 '2.0-acquire-and-prep',
 '2.00.01_Navigating_Excel.md',
 'notes',
 '10.1_Overview',
 'python',
 'query-and-results.html',
 'this.html',
 'for.html',
 'resume-portfolio',
 'asdfasdf',
 'capstone-workbook',
 'interview',
 'EDIT',
 'Dataset_Challenge',
 'Stats_Assessment',
 'this',
 'Regression%20_Project.md',
 'End_to_End_Clustering',
 'prep',
 'tables',
 '3.3-explore',
 'locations.html',
 'index.md',
 '51',
 '2.7-correlation',
 'writing-to-files.html',
 '10.10_Exercises',
 'grids',
 '132',
